In [1]:
# Excercise 2


In [2]:
import sys
from pathlib import Path
import numpy as np
import tensorflow as tf
import torch
from typing import Callable, Dict, Iterator, List, Optional, Sequence, Tuple, Union

Tensor = torch.Tensor
sys.path.insert(0, "..")


2022-06-13 16:45:17.147380: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-13 16:45:17.147422: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Haal de data op vanuit de functie in de gestructeerde folder indeling.

In [3]:
from src.data import make_dataset
from src.data import data_tools


In [4]:
dataset = make_dataset.get_eeg()


2022-06-13 16:45:20.501 | INFO     | src.data.make_dataset:get_eeg:27 - Data is downloaded to ../data/raw/datasets/eeg_data.


## Exercise 1
Na een verkenning van de data, komt naar voren dat de dataset uit 14 dimensies bestaat. De data wordt eerst ingeladen in chunks wanneer het label gelijk is. Hierbij wordt dus de data aan elkaar geclusterd. Dit wordt gedaan door middel van de class BaseDataset waarbij ik een functie ingebouwd heb waarbij de data wordt aangepast (process_data). Hierbij kijk ik eerst naar het huidige label (cur_label = index 14) en maak ik 2 lege lijsten aan. 1 lijst waar meerdere observaties (EEG_list) aan elkaar worden opgeslagen en 1 lijst waar alle chunks worden samengevoegd (chunks). Waarbij ik gebruik maak van een forloop en kijk telkens of het label gelijk is aan het huidige label en hierna plak ik alle observaties behalve index 14( label). Deze voeg ik toe aan de lege lijst (EEG_dim) en uiteindelijk aan de lijst buiten de forloop (EEG_list). Dit doe ik ook als het label niet gelijk is aan huidig label. Uiteindelijk voeg ik de lijsten weer bij elkaar in de list chunks.  

In [5]:
dataloader = data_tools.BaseDataset(datapath=dataset)


Hierbij komen 24 chunks naar voren.

In [6]:
dataloader.__len__()


24

Eerst is dus een label zichtbaar (bij item 0 > label 0) en de tensors. 

In [7]:
dataloader.__getitem__(0)


(0,
 tensor([[4329.2300, 4009.2300, 4289.2300,  ..., 4280.5098, 4635.8999,
          4393.8501],
         [4324.6201, 4004.6201, 4293.8501,  ..., 4279.4902, 4632.8198,
          4384.1001],
         [4327.6899, 4006.6699, 4295.3799,  ..., 4282.0498, 4628.7202,
          4389.2300],
         ...,
         [4468.2100, 4044.6201, 4305.1299,  ..., 4367.6899, 4833.8501,
          4571.7900],
         [4461.0298, 4041.0300, 4300.0000,  ..., 4365.1299, 4826.6699,
          4558.4600],
         [4452.8198, 4032.3101, 4295.3799,  ..., 4353.3301, 4808.2100,
          4549.2300]]))

In [8]:
x, y = dataloader[0]
x, y.shape


(0, torch.Size([188, 14]))

Hierbij zie je dus dat de eerste chunk uit een sequence lengte van 188 bij 14 dimensies bestaat met label 0. De tweede observatie een sequence lengte van 683 bij 14 dimensies heeft met label 1. We kunnen hierbij door middel van de window functie, telkens over elke observatie windowen en op die manier gelijke lengtes krijgen. Wel komen hier dan meerdere windows van elke observatie naar voren. De window functie heb ik aab de BaseDataIterator toegevoegd (class BaseDataIterator_wind). Hierbij heb ik als basis de window functie uit les 3 gebruikt en maak ik weer een lege lijst aan. Hierna maak ik  gebruik van een forloop waarbij ik door de 24 chunks van de dataset ga (len(self.dataset)). Ik kijk per chunk hoelang de sequence lengte is en wordt er gekeken hoevaak de window erdoor heen kan. Ik start met een window_size van 5.   

In [9]:
window_data = data_tools.BaseDataIterator_wind(dataloader, 5)


In [10]:
window_data.__len__()


24

In onderstaande uitdraai zie je het eerste item in combinatie met een window van 5, waarbij het tweede gedeelte dus ook met de tweede waarde begint (4324). 

In [11]:
window_data.__getitem__(0)


(0,
 tensor([[[4329.2300, 4009.2300, 4289.2300,  ..., 4280.5098, 4635.8999,
           4393.8501],
          [4324.6201, 4004.6201, 4293.8501,  ..., 4279.4902, 4632.8198,
           4384.1001],
          [4327.6899, 4006.6699, 4295.3799,  ..., 4282.0498, 4628.7202,
           4389.2300],
          ...,
          [4315.8999, 4003.5901, 4257.4399,  ..., 4270.2598, 4621.5400,
           4378.9702],
          [4314.3599, 4005.6399, 4259.4902,  ..., 4272.3101, 4624.6201,
           4381.5400],
          [4324.6201, 4012.8201, 4263.0801,  ..., 4274.3599, 4632.3101,
           4400.5098]],
 
         [[4324.6201, 4004.6201, 4293.8501,  ..., 4279.4902, 4632.8198,
           4384.1001],
          [4327.6899, 4006.6699, 4295.3799,  ..., 4282.0498, 4628.7202,
           4389.2300],
          [4328.7202, 4011.7900, 4296.4102,  ..., 4287.6899, 4632.3101,
           4396.4102],
          ...,
          [4314.3599, 4005.6399, 4259.4902,  ..., 4272.3101, 4624.6201,
           4381.5400],
          [43

In [12]:
x, y = window_data[0]
y.shape


torch.Size([167, 22, 14])

Hierbij is 184 de maximale hoeveelheid slices we vanuit de array [0][1] kunnen halen met een window van 5. 

## Exercise 2

Kijken naar de eerste observatie. Een window van 5 is klein, stel ik ga groter windowen dan kan het zijn dat sommige observaties te kort zijn. De maximale window is 22. Een window van 200 geeft bijvoorbeeld problemen. Om deze reden wordt padding toegevoegd waardoor je weet dat de observatie nooit te kort kan zijn. Deze heb ik in een aparte class (BaseDataIterator_pad) geschreven waarbij ik de functie padding gebruik. Hierbij maak ik weer een lege lijst aan waar ik met een forloop door de chunks (24) heen ga. Hierbij kijk ik telkens hoelang de sequence lengte is en bereken ik het verschil met de window size. Stel dit verschil is niet 0 dan wordt het verschil door middel van de F.pad functie toegevoegd. Als deze gelijk is aan 0 wordt er geen extra padding toegepast. 


In [13]:
data_padding = data_tools.BaseDataIterator_pad(dataloader, 200)


In [14]:
data_padding.__len__()


24

In [15]:
data_padding.__getitem__(0)


(0,
 tensor([[4329.2300, 4009.2300, 4289.2300,  ..., 4280.5098, 4635.8999,
          4393.8501],
         [4324.6201, 4004.6201, 4293.8501,  ..., 4279.4902, 4632.8198,
          4384.1001],
         [4327.6899, 4006.6699, 4295.3799,  ..., 4282.0498, 4628.7202,
          4389.2300],
         ...,
         [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
             0.0000],
         [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
             0.0000],
         [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
             0.0000]]))

In [16]:
x, y = data_padding[0]
y.shape


torch.Size([200, 14])

Hierbij zie je dus bij de eerste observatie dat de sequence lengte is uitgebreid met 12 (naar 200). De sequence lengte was eerst 188. Hierbij moet wel rekening gehouden dat bijvoorbeeld bij de observatie met een sequence lengte van 23 er heel veel padding is. De BaseDataIterator class kun je combineren met de class van de windowing door de data_padding in te voegen in de BaseDataIterator_wind (bv. data_test = data_tools.BaseDataIterator_wind(data_padding,200)). Echter heb ik de windowing en padding gecombineerd in één functie (padding_windowing) in de class BaseDataIterator_pad_wind. In de basis zijn de voorgaande functies van windowing en padding gelijk gebleven. Hierbij zie je nu ook dat de window_size groot kan zijn (bijvoorbeeld 200). 

In [28]:
data_ex2 = data_tools.BaseDataIterator_pad_wind(dataloader, 200)


In [29]:
data_ex2.__getitem__(0)


(0,
 tensor([[[4329.2300, 4009.2300, 4289.2300,  ..., 4280.5098, 4635.8999,
           4393.8501],
          [4324.6201, 4004.6201, 4293.8501,  ..., 4279.4902, 4632.8198,
           4384.1001],
          [4327.6899, 4006.6699, 4295.3799,  ..., 4282.0498, 4628.7202,
           4389.2300],
          ...,
          [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
              0.0000],
          [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
              0.0000],
          [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
              0.0000]]]))

In [31]:
x, y = data_ex2[0]
x, y.shape


(0, torch.Size([1, 200, 14]))